# Torchdiffeq with Deepchem

Author : [Anshuman Mishra](https://github.com/shivance)

In this tutorial, we will explore use neural ordinary differential equation model in our neural network ! 

We use deepchem's molnet dataset.

## Colab

This tutorial and the rest in this sequence are designed to be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1x1_EexmiTk01dsAbdfopWKWbIGENiXC1?usp=sharing)


### Installing Libraries

In [ ]:
!pip install torchdiffeq
!pip install --pre deepchem

### Import Libraries

In [1]:
import torch
import torch.nn as nn

from torchdiffeq import odeint
import math

import deepchem as dc

## Make dataset deepchem

In [5]:
tasks, dataset, transformers = dc.molnet.load_delaney(featurizer='ECFP', splitter='random')
train_set, valid_set, test_set = dataset
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

# Model

In [6]:
from torchdiffeq import odeint_adjoint as odeadj

class ODEfunction(nn.Module):

  def __init__(self, dim):
    super(ODEfunction, self).__init__()
    self.model = nn.Sequential(
        nn.Linear(dim,124),
        nn.ReLU(),
        nn.Linear(124,124),
        nn.ReLU(),
        nn.Linear(124,dim),
        nn.Tanh()
    )

  def forward(self, t, x):
    return self.model(x)

class ODEBlock(nn.Module):

  def __init__(self, odefunction):
    super(ODEBlock, self).__init__()
    self.odefunction = odefunction
    self.integration_time = torch.Tensor([0,1]).float()

  def forward(self, x):
    self.integration_time = self.integration_time.type_as(x)
    out = odeadj(
        self.odefunction,
        x,
        self.integration_time
    )

    return out[1]


class ODENet(nn.Module):

  def __init__(self, in_dim, mid_dim, out_dim):
    super(ODENet, self).__init__()
    odefunction = ODEfunction(dim=mid_dim)
    self.fc1 = nn.Linear(in_dim, mid_dim)
    self.relu1 = nn.ReLU(inplace=True)
    self.norm1 = nn.BatchNorm1d(mid_dim)
    self.ode_block = ODEBlock(odefunction)
    self.dropout = nn.Dropout(0.4)
    self.norm2 = nn.BatchNorm1d(mid_dim)
    self.fc2 = nn.Linear(mid_dim, out_dim)

  def forward(self, x):
    batch_size = x.shape[0]
    x = x.view(batch_size, -1)

    out = self.fc1(x)
    out = self.relu1(out)
    out = self.norm1(out)
    out = self.ode_block(out)
    out = self.norm2(out)
    out = self.dropout(out)
    out = self.fc2(out)

    return out

## Time to Train

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = ODENet(in_dim=1024, mid_dim=1000, out_dim=1).to(device)
model = dc.models.TorchModel(model, dc.models.losses.L2Loss())

model.fit(train_set, nb_epoch=5)

print('Training set score : ', model.evaluate(train_set,[metric]))
print('Test set score : ', model.evaluate(test_set,[metric]))

Training set score :  {'pearson_r2_score': 0.8971549648849394}
Test set score :  {'pearson_r2_score': 0.7401856633636557}


# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!